###  1: 导入库与加载数据

**代码分析:**

*   **集中导入**: 在 Notebook 的最开始，我们一次性导入所有需要用到的 Python 库，包括用于数据处理的 `pandas`、科学计算的 `numpy`，以及 `scikit-learn` 中用于特征工程、模型训练和评估的各个模块。这是一个良好的编程习惯，使得代码的依赖关系一目了然。
*   **数据加载**: 我们使用 `pd.read_csv()` 来加载竞赛提供的三个核心文件：`train.csv` (训练数据)，`test.csv` (测试数据)，和 `sample_submission.csv` (提交格式示例)。
*   **健壮性**: 整个加载过程被包裹在 `try...except FileNotFoundError` 块中。这样做的好处是，如果因为某种原因（例如数据集未添加到 Notebook）导致文件路径无效，程序不会直接崩溃，而是会打印出一条清晰的错误提示信息，便于我们快速定位问题。

In [1]:
# ===  1: 导入所有必需的库与加载数据 ===

# 数据处理与科学计算
import pandas as pd
import numpy as np

# 特征工程
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

# 模型与评估
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# 深度学习嵌入模型
from sentence_transformers import SentenceTransformer

print("🚀 Team 8 Baseline & Embedding Model Pipeline 启动！")

# --- 加载数据集 ---
# 使用 try-except 结构确保数据加载的健壮性
try:
    train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
    test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
    sample = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")
    print("✅ 数据集加载成功!")
    print(f"  训练集大小: {train.shape}")
    print(f"  测试集大小: {test.shape}")
except FileNotFoundError:
    print("❌ 数据加载失败! 请检查竞赛数据集是否已正确添加到 Notebook.")

2025-10-24 15:36:53.097051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761320213.285253      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761320213.336074      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🚀 Team 8 Baseline & Embedding Model Pipeline 启动！
✅ 数据集加载成功!
  训练集大小: (57477, 9)
  测试集大小: (3, 4)


###  2: 特征工程

**代码分析:**

*   **核心原则**: 为了保证模型的公平性和有效性，所有对训练集 `train` 进行的特征工程，都**必须以完全相同的方式**应用到测试集 `test` 上。这确保了模型在训练和预测时看到的数据结构是完全一致的，从而避免了 `KeyError` 等常见错误。

*   **1. 标签生成 (`label` - 仅限训练集)**:
    *   **作用**: 将训练数据中 `winner_model_a`, `winner_model_b`, `winner_tie` 这三列（one-hot 编码格式）转换为一个单一的、多分类的标签列。
    *   **方法**: `.argmax(axis=1)` 会沿着每一行查找最大值的索引。例如，如果 `winner_model_a` 是1，它的索引是0；如果 `winner_model_b` 是1，索引是1；如果是平局，索引是2。这样，我们就得到了一个适合分类模型训练的目标变量 `y` (值为 0, 1, 2)。

*   **2. 文本与长度特征工程 (针对 `train` 和 `test` 集)**:
    *   **文本拼接 (`text`)**: 我们将 `prompt` 与两个 `response` (`response_a`, `response_b`) 分别拼接，然后使用一个特殊的分隔符 `[SEP]` 将两个完整的响应文本连接起来。`[SEP]` 分隔符为模型提供了一个明确的边界，有助于模型更好地区分和比较两个不同的响应。
    *   **长度特征 (`..._len`)**: 我们提取了 `prompt`, `response_a`, 和 `response_b` 的字符长度作为额外的数值特征。文本长度本身在很多NLP任务中都是一个有用的信号。

In [2]:
# ===  2: 特征工程 (对 train 和 test 集进行一致性处理) ===

print("\n--- 正在执行 Step 1: 特征工程 ---")

# --- 1. 处理训练集 (train) ---
# 将 one-hot 编码的标签转换为单一的多分类标签 (0, 1, 2)
train['label'] = train[['winner_model_a', 'winner_model_b', 'winner_tie']].values.argmax(axis=1)

# 创建包含 prompt 和 response 的完整文本
train['text_a'] = train['prompt'] + " " + train['response_a']
train['text_b'] = train['prompt'] + " " + train['response_b']

# 使用 [SEP] 分隔符连接两个响应，便于模型对比
train['text'] = train['text_a'] + " [SEP] " + train['text_b']

# 计算长度特征
train['prompt_len'] = train['prompt'].str.len()
train['resp_a_len'] = train['response_a'].str.len()
train['resp_b_len'] = train['response_b'].str.len()


# --- 2. 用完全相同的方法处理测试集 (test) ---
# 注意：测试集没有 'label' 列，所以不需要处理
test['text_a'] = test['prompt'] + " " + test['response_a']
test['text_b'] = test['prompt'] + " " + test['response_b']
test['text'] = test['text_a'] + " [SEP] " + test['text_b']
test['prompt_len'] = test['prompt'].str.len()
test['resp_a_len'] = test['response_a'].str.len()
test['resp_b_len'] = test['response_b'].str.len()

print("✅ 特征工程完成。")


--- 正在执行 Step 1: 特征工程 ---
✅ 特征工程完成。


###  3: Baseline 模型 (词袋 + 逻辑回归)

**代码分析:**

*   **1. 向量化**:
    *   **文本向量化 (`CountVectorizer`)**: 我们使用“词袋模型”(Bag of Words)将文本数据转换为数值矩阵。`max_features=5000` 参数限制了只使用最常见的 5000 个词/词组作为特征，以控制维度。`ngram_range=(1,2)` 参数让模型同时考虑单个词（unigrams）和相邻的词对（bigrams），能捕捉到更丰富的短语信息。
    *   **数值特征标准化 (`StandardScaler`)**: 对我们之前创建的长度特征进行标准化处理，使其均值为0，方差为1。这一步至关重要，因为逻辑回归等模型对特征的尺度很敏感，标准化可以防止模型过分偏重于数值范围较大的特征。
    *   **特征合并 (`np.hstack`)**: 将处理好的文本特征矩阵和数值特征矩阵水平拼接，形成最终用于训练基线模型的完整特征矩阵 `X_baseline`。

*   **2. 训练与验证**:
    *   **数据划分 (`train_test_split`)**: 我们将特征矩阵和标签按 8:2 的比例划分为训练集和验证集。`random_state=42` 确保了每次划分的结果都一样，保证了实验的可复现性。
    *   **模型训练 (`LogisticRegression`)**: 我们选用逻辑回归作为基线分类器。它是一个简单、快速且解释性强的线性模型，非常适合作为项目的起点。我们设置 `max_iter=1000` 来增加最大迭代次数，以确保模型能够充分训练并收敛，避免 `ConvergenceWarning`。
    *   **性能评估**: 模型在训练集上训练后，我们在从未见过的验证集上进行预测，并使用竞赛的官方评估指标 `log_loss` 来计算验证分数。这个分数可以帮助我们在提交前快速评估模型的性能。

In [3]:
# ===  3: Baseline 模型 (词袋 + 逻辑回归) ===

print("\n--- 正在执行 Baseline 模型 ---")

# --- 1. 向量化 ---
# 文本特征 (Bag of Words)，考虑 unigrams 和 bigrams
vectorizer = CountVectorizer(max_features=5000, ngram_range=(1,2))
X_text_train = vectorizer.fit_transform(train['text'])
X_text_test = vectorizer.transform(test['text'])

# 数值特征 (长度)，并进行标准化
scaler = StandardScaler()
num_features_train = train[['prompt_len','resp_a_len','resp_b_len']]
num_features_test = test[['prompt_len','resp_a_len','resp_b_len']]
X_num_train = scaler.fit_transform(num_features_train)
X_num_test = scaler.transform(num_features_test)

# 合并文本特征和数值特征
X_baseline = np.hstack([X_text_train.toarray(), X_num_train])
X_test_baseline = np.hstack([X_text_test.toarray(), X_num_test])
y = train['label']

# --- 2. 训练与验证 ---
# 将基线模型的特征集划分为训练集和验证集
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(
    X_baseline, y, test_size=0.2, random_state=42
)

# 训练逻辑回归模型，增加 max_iter 以避免收敛警告
clf_base = LogisticRegression(max_iter=1000)
clf_base.fit(X_train_base, y_train_base)

# 在验证集上评估模型性能
y_pred_val_base = clf_base.predict_proba(X_val_base)
validation_score_base = log_loss(y_val_base, y_pred_val_base)
print(f"📊 Validation LogLoss (Baseline): {validation_score_base:.5f}")

# 注意：我们不再从此模型生成 submission.csv，最终提交文件将由更强的模型生成。


--- 正在执行 Baseline 模型 ---
📊 Validation LogLoss (Baseline): 1.28668


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 单元格 4: Embedding 模型 (MiniLM)

**代码分析:**

*   **1. 模型加载 (`SentenceTransformer`)**:
    *   **作用**: 我们加载一个预训练的 Transformer 模型 `all-MiniLM-L6-v2`。与词袋模型不同，它能够将整个句子转换为一个固定长度的、包含丰富语义信息的向量（即“嵌入”或 Embedding）。
    *   **离线模式**: 为了满足竞赛的可复现性要求（通常要求关闭网络），我们从预先添加到 Notebook 的 Kaggle 数据集路径 (`/kaggle/input/...`) 加载模型，而不是从网络下载。`device='cuda'` 参数指定使用 GPU 进行计算，可以极大地加速后续的编码过程。

*   **2. 生成句向量 (`model.encode`)**:
    *   **作用**: 这是核心步骤。我们将每个样本的文本输入到 MiniLM 模型中，输出一个固定维度（对于 MiniLM 是 384 维）的向量。
    *   **对比**: 这里的特征不再是词语的出现次数，而是深层语义的向量表示。这通常会带来比词袋模型显著的性能提升。`batch_size=128` 参数允许模型在 GPU 上进行批量处理，进一步提升了编码效率。

*   **3. 训练与验证**:
    *   **数据划分**: 与 Baseline 模型类似，我们同样将生成的句向量 `train_emb` 划分为训练集和验证集，以便进行公平的性能比较。
    *   **模型训练**: 我们再次使用逻辑回归分类器，但这次的输入特征是高质量的句向量。
    *   **性能评估**: 我们计算并打印出 Embedding 模型在验证集上的 `log_loss` 分数。通过对比这个分数和 Baseline 模型的验证分数，我们可以量化地评估出从词袋模型升级到嵌入模型所带来的性能提升。

*   **4. 生成最终提交文件**:
    *   **预测**: 使用训练好的 Embedding 模型对测试集的句向量 `test_emb` 进行预测。
    *   **文件生成**: 将预测出的概率保存为 `submission.csv` 文件。**文件名必须是 `submission.csv`**，以符合 Kaggle 竞赛的提交要求。这个文件将覆盖掉之前可能存在的任何同名文件，确保最终提交的是我们更强的 Embedding 模型的结果。

In [4]:
# ===  4: Embedding 模型 (MiniLM + 逻辑回归) ===

print("\n--- 正在执行 Embedding 模型 ---")

# --- 1. 加载预训练的 SentenceTransformer 模型 (离线模式) ---
# 确保你已经在 Notebook 的 Input 中添加了 'sentence-transformers-all-minilm-l6-v2' 数据集
model_path = '/kaggle/input/sentencetransformersallminilml6v2'

model = None
try:
    model = SentenceTransformer(model_path, device='cuda') # 使用 GPU 加速
    print("✅ Embedding 模型加载成功！")
except Exception as e:
    print(f"❌ Embedding 模型加载失败: {e}")
    print("  请确保右侧 Input 面板中已添加 'sentencetransformersallminilml6v2' 数据集，并且路径正确。")


# 只有在模型成功加载后，才继续执行
if model is not None:
    # --- 2. 生成句向量 ---
    # 为训练和测试数据创建一个新的合并字段用于嵌入
    train['combined_for_embedding'] = train['prompt'] + " " + train['response_a'] + " [SEP] " + train['response_b']
    test['combined_for_embedding'] = test['prompt'] + " " + test['response_a'] + " [SEP] " + test['response_b']
    
    print("⏳ 正在为训练集生成句向量 (这可能需要几分钟)...")
    train_emb = model.encode(train['combined_for_embedding'].tolist(), show_progress_bar=True, batch_size=128)
    
    print("⏳ 正在为测试集生成句向量...")
    test_emb = model.encode(test['combined_for_embedding'].tolist(), show_progress_bar=True, batch_size=128)
    print("✅ 句向量生成完成。")

    # --- 3. 训练与验证 ---
    # 将嵌入特征集划分为训练集和验证集
    X_train_emb, X_val_emb, y_train_emb, y_val_emb = train_test_split(
        train_emb, y, test_size=0.2, random_state=42
    )

    # 训练逻辑回归分类器
    print("⏳ 正在训练 Embedding 模型的分类器...")
    clf_emb = LogisticRegression(max_iter=1000)
    clf_emb.fit(X_train_emb, y_train_emb)
    print("✅ 分类器训练完成。")

    # 在验证集上评估模型性能
    y_pred_val_emb = clf_emb.predict_proba(X_val_emb)
    validation_score_emb = log_loss(y_val_emb, y_pred_val_emb)
    print(f"📊 Validation LogLoss (Embedding): {validation_score_emb:.5f}")

    # --- 4. 生成最终的 Kaggle 提交文件 ---
    # 使用在部分数据上训练的模型进行预测
    # (更优的做法是用全部数据重新训练，但为了速度和简洁，这里直接使用 clf_emb)
    print("⏳ 正在为测试集生成最终预测...")
    preds_final = clf_emb.predict_proba(test_emb)

    # 创建提交 DataFrame，确保文件名为 "submission.csv"
    submission_final = pd.DataFrame(preds_final, columns=sample.columns[1:])
    submission_final.insert(0, "id", sample["id"])
    submission_final.to_csv("submission.csv", index=False)

    print("\n🎉 最终的 submission.csv 已生成！可以保存并提交了。")


--- 正在执行 Embedding 模型 ---


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

✅ Embedding 模型加载成功！
⏳ 正在为训练集生成句向量 (这可能需要几分钟)...


Batches:   0%|          | 0/450 [00:00<?, ?it/s]

⏳ 正在为测试集生成句向量...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 句向量生成完成。
⏳ 正在训练 Embedding 模型的分类器...
✅ 分类器训练完成。
📊 Validation LogLoss (Embedding): 1.08496
⏳ 正在为测试集生成最终预测...

🎉 最终的 submission.csv 已生成！可以保存并提交了。


| 模型 (Model) | Kaggle 公开分数 (Public Score) | 验证集 LogLoss (Validation LogLoss) | 
| :--- | :--- | :--- | 
| Baseline (词袋+逻辑回归) | `1.29503` | `1.28668` | 
| Embedding (MiniLM) | `1.08498` | `1.08496` (请替换为你的真实值) | 